In [1]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np

In [3]:
from IPython.display import clear_output

In [5]:
import numpy as np
from time import sleep
import clickhouse_connect

client = clickhouse_connect.get_client(
    host='localhost',       
    port=8123,              
    username='default',     
    password='',            
    database='krisha'      
)

In [7]:
numerical_keys=["price", "room_size",'year_built','apartment_floor','total_floor_of_house']
floats=["square_meters",  'kitchen_meters']

In [9]:
def convert(data):
    for row in data:
        for key in numerical_keys:
            try:
                row[key] = int(row[key]) if row.get(key) is not None else 0.0
            except (ValueError, TypeError):
                row[key] = 0
        for key in floats:
            try:
                row[key] = float(row[key]) if row.get(key) is not None else 0.0
            except (ValueError, TypeError):
                row[key] = 0.0
    
    # for i, row in enumerate(data):
    #     print(f"\nRow {i + 1}")
    #     for key, value in row.items():
    #         print(f"  {key}: {value} ({type(value).__name__})")
    return data

In [11]:
HEADERS = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'})

In [13]:
def get_apartment_links(page_html):
    soup = BeautifulSoup(page_html, "html.parser")
    links = []
    for a in soup.select("a.a-card__title"):  # CSS selector for listing titles
        href = a.get("href")
        if href:
            links.append("https://krisha.kz" + href)
    return links

In [15]:
def extract_apartment_data(detail_html):
    soup = BeautifulSoup(detail_html, "html.parser")

    # --- Price ---
    price_container = soup.find("div", class_="offer__price") or soup.find("p", class_="offer__price")
    if price_container:
        for span in price_container.find_all("span"):
            span.decompose()
        raw_price = price_container.get_text(strip=True)
        price = re.sub(r"[^\d]", "", raw_price)
    else:
        price = None

    # --- Address ---
    address = soup.find("div", attrs={"class" : 'offer__location offer__advert-short-info'})
    address = address.find("span").get_text(strip=True) if address and address.find("span") else None

    # --- Split Address ---
    city, province = None, None
    if address and "," in address:
        parts = address.split(",", 1)
        city = parts[0].strip()
        province = parts[1].strip()
    elif address:
        city = address.strip()

    # --- Room Size (from h1) ---
    room_title = soup.find("div", class_="offer__advert-title")
    if room_title:
        h1 = room_title.find("h1")
        if h1:
            text = h1.get_text(strip=True)
            room_match = re.search(r"(\d+)", text)
            room_size = room_match.group(1) if room_match else None
        else:
            room_size = None
    else:
        room_size = None

    # --- Square Meters & Kitchen Meters ---
    raw_area = None
    square_meters = None
    kitchen_meters = None
    square_container = soup.find("div", attrs={"data-name": "live.square"})
    if square_container:
        info = square_container.find("div", class_="offer__advert-short-info")
        if info:
            raw_area = info.get_text(strip=True)
            total_match = re.search(r"(\d+\.?\d*)\s*м²", raw_area)
            kitchen_match = re.search(r"Площадь кухни\s*—\s*(\d+\.?\d*)", raw_area)
            if total_match:
                square_meters = total_match.group(1)
            if kitchen_match:
                kitchen_meters = kitchen_match.group(1)

    # --- Helpers ---
    def safe_extract(selector):
        container = soup.find("div", attrs={"data-name": selector})
        if container:
            info = container.find("div", class_="offer__advert-short-info")
            if info:
                return info.get_text(strip=True)
        return None

    def safe_extract_text_from_a(selector):
        container = soup.find("div", attrs={"data-name": selector})
        if container:
            a_tag = container.find("a")
            if a_tag:
                return a_tag.get_text(strip=True)
        return None

    def safe_find_text_by_class(class_name):
        element = soup.find("div", class_=class_name)
        return element.get_text(strip=True) if element else None

    def safe_dt_dd(name):
        dt = soup.find("dt", attrs={"data-name": name})
        if dt:
            dd = dt.find("dd")
            if dd:
                return dd.get_text(strip=True)
        return None

    # --- Custom Fields ---
    year_built = safe_extract("house.year")
    house_type = safe_extract("flat.building")
    complex_name = safe_extract_text_from_a("map.complex")
    toilet_type = safe_extract("flat.toilet")
    floor_raw = safe_extract("flat.floor")
    parking = safe_extract("flat.parking")
    mortgaged = safe_find_text_by_class("offer__parameters-mortgaged")
    ceiling_height = safe_dt_dd("ceiling")

    # --- Split Floor ---
    apartment_floor, total_floor_of_house = None, None
    if floor_raw:
        match = re.search(r"(\d+)\s+из\s+(\d+)", floor_raw)
        if match:
            apartment_floor = int(match.group(1))
            total_floor_of_house = int(match.group(2))

    # --- Return Result ---
    return {
        "price": price,
        "city": city,
        "province": province,
        "room_size": room_size,
        "square_meters": square_meters,
        "kitchen_meters": kitchen_meters,
        "year_built": year_built,
        "house_type": house_type,
        "complex_name": complex_name,
        "toilet_type": toilet_type,
        "apartment_floor": apartment_floor,
        "total_floor_of_house": total_floor_of_house,
        "parking": parking,
        "mortgaged": mortgaged,
        "ceiling_height": ceiling_height,
    }


In [17]:
def scrape_all_apartments(base_url, max_pages=5, delay=1):
    all_data = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}?page={page}"
        print(f"Fetching listings from: {url}")
        response = requests.get(url)
        if not response.ok:
            print(f"Failed to fetch page {page}")
            break

        links = get_apartment_links(response.text)

        for link in links:
            print(f" → Parsing: {link}")
            detail_resp = requests.get(link)
            if detail_resp.ok:
                data = extract_apartment_data(detail_resp.text)
                data["url"] = link
                all_data.append(data)
            else:
                print(f"   ✗ Failed to load: {link}")
            sleep(delay)  # to avoid getting blocked

    return all_data


In [19]:
# base_url = "https://krisha.kz/prodazha/kvartiry/astana/"
# data = scrape_all_apartments(base_url, max_pages=1)
# data = convert(data)
# df = pd.json_normalize(data)
# for col in df.select_dtypes(include='object').columns:
#     df[col] = df[col].fillna('')
# df
# client.insert_df('krisha_apartments', df)

In [21]:
def pipeline(base_url, max_pages):
    data = scrape_all_apartments(base_url, max_pages=max_pages)

    existing_urls = set(
        row[0] for row in client.query("SELECT url FROM krisha_apartments").result_rows
    )

    # data = [row for row in data if row.get("url") not in existing_urls]
    
    data = convert(data)
    df = pd.json_normalize(data)
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].fillna('')

    
    if not df.empty:
        client.insert_df('krisha_apartments', df)
        print(f"✅ Inserted {len(df)} new rows")
    else:
        print("⚠️ No new rows to insert")
    return df

In [24]:
def pipeline(base_url, max_pages):
    existing_urls = set(
        row[0] for row in client.query("SELECT url FROM krisha_apartments").result_rows
    )

    inserted_total = 0

    for page in range(207, max_pages + 1):
        print(f"📄 Processing page {page}...")
        url = f"{base_url}?page={page}"
        data = scrape_all_apartments(url, max_pages=1)  # force it to scrape only this one page

        # Filter out existing URLs
        data = [row for row in data if row.get("url") not in existing_urls]

        if not data:
            print("⚠️ No new listings on this page.")
            continue

        # Convert types
        data = convert(data)

        # Normalize to DataFrame
        df = pd.json_normalize(data)

        # Fill empty strings in string columns
        for col in df.select_dtypes(include='object').columns:
            df[col] = df[col].fillna('')

        # Insert into ClickHouse
        if not df.empty:
            client.insert_df('krisha_apartments', df)
            inserted_total += len(df)
            clear_output(wait=True)
            print(f"✅ Inserted {len(df)} rows from page {page}")

            # Update existing_urls set to avoid future duplicates in this run
            existing_urls.update(df['url'].tolist())
        else:
            print(f"⚠️ No rows to insert from page {page}")

    print(f"\n🚀 Done! Inserted a total of {inserted_total} new rows.")


In [26]:
base_url = "https://krisha.kz/prodazha/kvartiry/astana/"
pipeline(base_url, 1000)

📄 Processing page 207...
Fetching listings from: https://krisha.kz/prodazha/kvartiry/astana/?page=207?page=1
⚠️ No new listings on this page.
📄 Processing page 208...
Fetching listings from: https://krisha.kz/prodazha/kvartiry/astana/?page=208?page=1
⚠️ No new listings on this page.
📄 Processing page 209...
Fetching listings from: https://krisha.kz/prodazha/kvartiry/astana/?page=209?page=1
⚠️ No new listings on this page.
📄 Processing page 210...
Fetching listings from: https://krisha.kz/prodazha/kvartiry/astana/?page=210?page=1
⚠️ No new listings on this page.
📄 Processing page 211...
Fetching listings from: https://krisha.kz/prodazha/kvartiry/astana/?page=211?page=1
⚠️ No new listings on this page.
📄 Processing page 212...
Fetching listings from: https://krisha.kz/prodazha/kvartiry/astana/?page=212?page=1
⚠️ No new listings on this page.
📄 Processing page 213...
Fetching listings from: https://krisha.kz/prodazha/kvartiry/astana/?page=213?page=1
⚠️ No new listings on this page.
📄 Proc

ConnectTimeout: HTTPSConnectionPool(host='krisha.kz', port=443): Max retries exceeded with url: /prodazha/kvartiry/astana/?page=295?page=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001DE07AFEC00>, 'Connection to krisha.kz timed out. (connect timeout=None)'))